### Set Up Packages and Authorize Access to GEE and Google Drive

In [ ]:
import importlib
import subprocess

# The list of libraries required
libraries = ["ee", "itertools", "jenkspy"]

# Automatically import/install the libraries
for i in libraries:
    try:
        library = importlib.import_module(i)
        print(f"Successfully imported {i}")
    except ImportError:
        print(f"{i} is not installed. Attempting to install it...")
        subprocess.check_call(["pip", "install", i])

        try:
            library = importlib.import_module(i)
            print(f"Successfully imported {i}")
        except ImportError:
            print(f"Failed to install and import {i}. Please install it manually.")

# Initialize GEE's Python API
import ee
ee.Authenticate()
ee.Initialize()

# Authorize access to Google Drive (for exporting data)
from google.colab import drive
drive.mount('/content/drive')

Successfully imported ee
Successfully imported itertools
jenkspy is not installed. Attempting to install it...
Successfully imported jenkspy
To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=pxi_peD5bLv4d6QaRvIzRUf7QMtDYzxem0sLxaHpcRk&tc=7GKB3LUjkLdLokso8wzfPMDDxBQqCX8BN2bAf6dCQZI&cc=M0dfcDzf-zunptiPhQiMf1jci9DoWu71yWGr8PLu3Ds

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXlNXpZKsgQQma172GdaYxJLRdMEZviDqzR3KvCAaj12VC_7ThSOPh0

Successfully saved authorization token.
Mounted at /content/drive


### Specify the GRES ID of the Reservoir(s) to be processed

In [ ]:
# Enter the list of GRES_ID you want to export the raster only (no max/min water polygons)
GRES_ID = [122]

# Where to export the raster on Google Drive?
RasterFolder = "Reservoir_Raster"

# Where to grab the ee.Image (pre-processed raster exported from Script 1)?
Asset_Folder_Path= 'projects/ee-climatechangehydrology/assets/GRes_ProbabilityRaster/'

# Start the export with looping
for id in GRES_ID:
  try:
    # Get the pre-processed probability raster (ee.Image)
    ProbabilityRaster = ee.Image(Asset_Folder_Path + 'GRES_' + str(id))
    # Re-scale the image with a min/max stretch (min: 0, max: 1)
    Image_MinMax = ProbabilityRaster.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry= ProbabilityRaster.geometry(),
        scale=10,
        maxPixels= 1e13,
        tileScale = 16
    )
    min = Image_MinMax.getNumber('probability_min')
    max = Image_MinMax.getNumber('probability_max')
    norm_prob_ras = ProbabilityRaster.select('probability').unitScale(min,max)

    # Initiate export tasks
    Export_ProbailityRaster = ee.batch.Export.image.toDrive(
        image = norm_prob_ras,
        region = norm_prob_ras.geometry(),
        folder = RasterFolder,
        description = 'GRES_' + str(id) + '_GeoTIFF',
        fileFormat = 'GeoTIFF',
        scale = 10,
        maxPixels = 1e13
    )

    Export_ProbailityRaster.start()

  except Exception as e:
    print(f"Operation Failed on GRES_ID: {id}. Please check your input.")

# Print when execution completes
print("Operation is completed.")

Operation is completed. Check your export tasks on GEE's Task Manager
